In [ ]:
import dotenv
import logging
import sys
import uuid
from pprint import pprint

dotenv.load_dotenv()

logging.disable(sys.maxsize)

In [ ]:
DIGICOACH_NAME = "Digicoach test"
TASK_NAME = "Taak test"

In [43]:
from askdelphi_client import AskDelphiClient

client = AskDelphiClient()

client.authenticate()

True

In [40]:
try:
    topic_type_map = {}

    design = client.get_content_design()

    topic_types = design.get("topicTypes", [])
    print(f"Found {len(topic_types)} topic types:\n")

    for i, tt in enumerate(topic_types[:100]):  # Show first 100
        if tt.get("title").lower() in ["action", "task", "digitale coach procespagina"]:
            print(f"  {i+1}. {tt.get('title', 'Unknown')}")
            print(f"     ID: {tt.get('key', 'N/A')}")
            print(f"     Namespace: {tt.get('namespace', 'N/A')}")
            print()

            topic_type_map[tt.get("title")] = tt.get("key")

    if len(topic_types) > 100:
        print(f"  ... and {len(topic_types) - 100} more")

except Exception as e:
    print(f"FAILED: {e}")

print(f"topic type map:\n")
pprint(topic_type_map)

Found 39 topic types:

  21. Task
     ID: 6aba8437-c8df-42d2-a868-840847c124ca
     Namespace: http://tempuri.org/imola-task

  22. Digitale Coach Procespagina
     ID: 7d332fbb-44f5-469f-b570-874e701e526b
     Namespace: http://tempuri.org/imola-skill-area

  29. Action
     ID: c568af9a-6c89-45cf-a580-bc94e1c62ae3
     Namespace: http://tempuri.org/imola-action

topic type map:

{'Action': 'c568af9a-6c89-45cf-a580-bc94e1c62ae3',
 'Digitale Coach Procespagina': '7d332fbb-44f5-469f-b570-874e701e526b',
 'Task': '6aba8437-c8df-42d2-a868-840847c124ca'}


In [ ]:
# create digicoach
topic_id1 = str(uuid.uuid4())

result_1 = client._request(
    "POST",
    "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic",
    json_data={
        "topicId": topic_id1,
        "topicTitle": DIGICOACH_NAME,
        "topicTypeId": topic_type_map["Digitale Coach Procespagina"],
        }
)

{'topicId': '6a656018-591b-4b2f-adf4-5e00d216678f', 'topicVersionKey': '029ecb8d-1070-4074-9da8-25c6a4f24520'}


In [ ]:
# get digicoach topic version and check out topic
endpoint = f"v1/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id1}/workflowstate"
result = client._request("POST", endpoint,
                json_data={
                    "action": 1
                })
topicVersionId = result["topicVersionId"]
print(topicVersionId)

4b1cdd4b-c528-492d-bc99-565d54c628a7


In [ ]:
# get relationship ID for task
task_relation_id = ""
endpoint = f"/v2/tenant/{{tenantId}}/project/{{projectId}}/acl/{{aclEntryId}}/topic/{topic_id1}/topicVersion/{topicVersionId}/allowedrelations"
result = client._request("GET", endpoint,
                json_data={}
                )
for relation in result["topicAllowedRelations"]:
    if relation["topicTypeName"] == "Taak":
        task_relation_id = relation["relationTypeId"]
        break

task_relation_id



In [ ]:
topic_id2 = str(uuid.uuid4())

result_2 = client._request(
    "POST",
    "v4/tenant/{tenantId}/project/{projectId}/acl/{aclEntryId}/topic",
    json_data={
        "topicId": topic_id2,
        "topicTitle": TASK_NAME,
        "topicTypeId": topic_type_map["Task"],
        "parentTopicId": topic_id1,
        "parentTopicRelationTypeId": task_relation_id,
        "parentTopicVersionId": topicVersionId
        }
)

{'topicId': 'f7dbff88-4a75-47c0-b365-c03180a3cc72', 'topicVersionKey': '21d285bc-03f6-4c9d-88f6-4c5a9dda6d91'}
